# Optimización de Pesos con Umbral (Objetivo Híbrido + Métricas de Pérdida)

Este notebook implementa el procedimiento completo de optimización de pesos.
Incluye cálculo de **FPR** y **Valor de la Función Objetivo** (Loss) para comparar configuraciones.

## Características
1.  **Objetivo Híbrido**: Minimiza error cuadrático pero castiga x5 los casos de "Ganar sin merecerlo".
2.  **Configuración Simplificada**: Definición de items por tipo (`easy`, `filter`, `advanced`, `exam`).
3.  **Comparativa Completa**: Muestra FPR y Loss (Pérdida) para la configuración original y la optimizada.


In [252]:
import numpy as np
import pandas as pd
import copy
from scipy.optimize import minimize, Bounds, LinearConstraint

np.random.seed(42)
pd.set_option('display.float_format', '{:.2f}'.format)
pd.set_option('display.max_columns', None)


In [253]:
# 1. CONFIGURACIÓN DE EVALUACIÓN
# Tipos: 'easy', 'filter', 'advanced', 'exam'.

config_evaluacion = {
    "regular_items": [
        # MÓDULO 1: Intro (Easy)
        {
            "name": "Quiz_1", 
            "weakness": 3.5,
            "correlated_with": "Tarea_1", "correlation_factor": 0.5,
            "type": "easy" 
        },
        {
            "name": "Tarea_1", 
            "weakness": 2.5,
            "min": 0.05, "max": 0.20, "suggested": 0.15, 
            "type": "advanced"
        },
        
        # MÓDULO 2: Filtro (Filter) - Correlacionado con M1
        {
            "name": "Quiz_2", 
            "correlated_with": "Quiz_1", "correlation_factor": 1.2,
            "weakness": 3.5,
            "type": "easy"
        },
        {
            "name": "Tarea_2", 
            "correlated_with": "Tarea_1", "correlation_factor": 1.2,
            "weakness": 2.5,
            "type": "filter"
        },
        
        # MÓDULO 3: Avanzado (Normal) - Correlacionado con M1 (Factor 1.0)
        {
            "name": "Quiz_3", 
            "correlated_with": "Quiz_1", "correlation_factor": 1.0,
            "weakness": 3.5,
            "type": "easy"
        },
        {
            "name": "Tarea_3", 
            "weakness": 2.5,
            "correlated_with": "Tarea_1", "correlation_factor": 1.0,
            "type": "advanced"
        },
    ],
    "definitory_item": {
        "name": "Examen Final",
        "sudden_death": 2.50,
        "max_weight": 0.40,
        "type": "exam"
    }
}

# --- AUTO-AJUSTE PARA CORRELACIONADOS ---
print("Autoconfigurando items correlacionados...")
reg_items = config_evaluacion['regular_items']
item_map = {item['name']: item for item in reg_items}

for item in reg_items:
    if 'correlated_with' in item:
        parent_name = item['correlated_with']
        factor = item['correlation_factor']
        if parent_name in item_map:
            parent = item_map[parent_name]
            item['peso_min'] = parent.get('peso_min', 0.0) * factor
            item['peso_max'] = parent.get('peso_max', 1.0) * factor
            item['peso_sugerido'] = parent.get('peso_sugerido', 0.0) * factor
            print(f" > {item['name']}: Ajustado vs {parent_name} (x{factor})")
        else:
            print(f" [!] Error: Padre '{parent_name}' no encontrado para '{item['name']}'")

print("\nResumen Configuración:")
for item in config_evaluacion['regular_items']:
    rel = f"(Depende de {item['correlated_with']})" if 'correlated_with' in item else "(Independiente)"
    print(f" - {item['name']:<10} Type='{item.get('type','?')}' {rel}")
print(f" - {config_evaluacion['definitory_item']['name']:<10} Type='{config_evaluacion['definitory_item'].get('type','?')}'")


Autoconfigurando items correlacionados...
 > Quiz_1: Ajustado vs Tarea_1 (x0.5)
 > Quiz_2: Ajustado vs Quiz_1 (x1.2)
 > Tarea_2: Ajustado vs Tarea_1 (x1.2)
 > Quiz_3: Ajustado vs Quiz_1 (x1.0)
 > Tarea_3: Ajustado vs Tarea_1 (x1.0)

Resumen Configuración:
 - Quiz_1     Type='easy' (Depende de Tarea_1)
 - Tarea_1    Type='advanced' (Independiente)
 - Quiz_2     Type='easy' (Depende de Quiz_1)
 - Tarea_2    Type='filter' (Depende de Tarea_1)
 - Quiz_3     Type='easy' (Depende de Quiz_1)
 - Tarea_3    Type='advanced' (Depende de Tarea_1)
 - Examen Final Type='exam'


In [254]:
# 2. GENERACIÓN DE DATOS (Por Tipo)

def generar_distribucion_bimodal(n, low_center=1.5, high_center=4.5, ratio=0.5, sigma=0.6):
    n_high = int(n * ratio)
    n_low = n - n_high
    high_grades = np.random.normal(high_center, sigma, n_high)
    low_grades = np.random.normal(low_center, sigma, n_low)
    combined = np.concatenate([high_grades, low_grades])
    np.random.shuffle(combined)
    return np.clip(combined, 0, 5)

def generar_distribucion_negative_skew(n, mode=4.5, sigma=1.0):
    raw = mode - np.random.exponential(scale=0.8, size=n)
    return np.clip(raw, 0, 5)

def generar_distribucion_normal(n, mean=3.0, sigma=1.0):
    return np.clip(np.random.normal(mean, sigma, n), 0, 5)

def generar_distribucion_exam(n, mean=2.8, sigma=1.1):
    return np.clip(np.random.normal(mean, sigma, n), 0, 5)

def generate_data(config, N=2000):
    data = {}
    
    def generate_column_by_type(dtype):
        if dtype == 'filter':
            return generar_distribucion_bimodal(N, low_center=1.2, high_center=4.0, ratio=0.4)
        elif dtype == 'easy':
            return generar_distribucion_negative_skew(N, mode=4.5)
        elif dtype == 'exam':
            return generar_distribucion_exam(N, mean=2.8, sigma=1.1)
        else: # advanced / normal
            return generar_distribucion_normal(N, mean=3.0, sigma=1.0)

    # Regular Items
    for item in config['regular_items']:
        dtype = item.get('type', 'advanced')
        data[item['name']] = generate_column_by_type(dtype)
        
    # Definitory Item
    def_item = config['definitory_item']
    dtype = def_item.get('type', 'advanced')
    data[def_item['name']] = generate_column_by_type(dtype)
    
    df = pd.DataFrame(data)
    df = df.sample(frac=1, random_state=42).reset_index(drop=True)
    return df.round(2)

df_notas = generate_data(config_evaluacion)
print(f"Datos generados: {df_notas.shape}")
display(df_notas.describe())


Datos generados: (2000, 7)


,Quiz_1,Tarea_1,Quiz_2,Tarea_2,Quiz_3,Tarea_3,Examen Final
count,2000.00,2000.00,2000.00,2000.00,2000.00,2000.00,2000.00
mean,3.70,3.00,3.70,2.29,3.68,2.98,2.82
std,0.80,0.98,0.79,1.47,0.80,0.98,1.09
min,0.00,0.00,0.00,0.00,0.00,0.00,0.00
25%,3.39,2.34,3.39,1.05,3.35,2.31,2.08
50%,3.93,3.00,3.96,1.81,3.94,2.98,2.85
75%,4.28,3.68,4.27,3.77,4.26,3.65,3.59
max,4.50,5.00,4.50,5.00,4.50,5.00,5.00


In [255]:
# 3. FUNCIONES AUXILIARES DE PESOS (Correlación)

def get_independent_indices(config):
    indices = []
    items = config['regular_items']
    for i, item in enumerate(items):
        if 'correlated_with' not in item:
            indices.append(i)
    return indices

def reconstruct_full_weights(x_independent, config):
    items = config['regular_items']
    indep_indices = get_independent_indices(config)
    
    # Map name -> weight
    weight_map = {}
    
    # 1. Fill Independent Weights
    current_indep_idx = 0
    full_weights = np.zeros(len(items))
    
    # First pass: Set independent weights to map
    for i in indep_indices:
        val = x_independent[current_indep_idx]
        item_name = items[i]['name']
        weight_map[item_name] = val
        full_weights[i] = val
        current_indep_idx += 1
        
    # 2. Fill Dependent Weights
    for i, item in enumerate(items):
        if 'correlated_with' in item:
            parent = item['correlated_with']
            factor = item['correlation_factor']
            if parent in weight_map:
                w_val = weight_map[parent] * factor
                full_weights[i] = w_val
                weight_map[item['name']] = w_val # Store for chain dependencies (if any)
            else:
                raise ValueError(f"Parent item '{parent}' not found or not yet processed for '{item['name']}'")
                
    return full_weights


In [256]:
# 4. ORÁCULO DINÁMICO
def get_oracle_decisions_general(weights_regular, df, config):
    reg_items = config['regular_items']
    def_item = config['definitory_item']
    
    w_sum_reg = np.sum(weights_regular)
    w_def = 1.0 - w_sum_reg
    
    reg_names = [item['name'] for item in reg_items]
    def_name = def_item['name']
    
    # Scores
    reg_scores = df[reg_names].values
    score_regular = np.sum(reg_scores * weights_regular, axis=1) / (w_sum_reg + 1e-9)
    score_def = df[def_name].values
    
    decisions = np.ones(len(df), dtype=int)
    
    # 1. Sudden Death
    decisions[score_def < def_item['sudden_death']] = 0
    
    # 2. Weakness
    weakness_thresholds = np.array([item['weakness'] for item in reg_items])
    agg_weakness_thresh = np.sum(weakness_thresholds * weights_regular) / (w_sum_reg + 1e-9)
    
    mask_weak = (score_def >= def_item['sudden_death']) &                 (score_def < 3.0) &                 (score_regular < agg_weakness_thresh)
    decisions[mask_weak] = 0
    
    return decisions


In [261]:
# 5. CÁLCULO DE NOTAS Y DETALLES (U-FACTORS)
def calculate_grade_strict_general(weights_regular, df, config, U_val=3.0, return_details=False):
    reg_items = config['regular_items']
    def_item = config['definitory_item']
    
    w_sum_reg = np.sum(weights_regular)
    w_def = 1.0 - w_sum_reg
    all_weights = np.concatenate([weights_regular, [w_def]])
    
    reg_names = [item['name'] for item in reg_items]
    def_name = def_item['name']
    all_names = reg_names + [def_name]
    
    grades_matrix = df[all_names].values
    
    # U-Factors
    ratios = np.clip(grades_matrix / U_val, 0, 1)
    n_items = len(all_names)
    u_factors = np.zeros_like(grades_matrix)
    
    for i in range(n_items):
        mask = np.ones(n_items, dtype=bool)
        mask[i] = False
        u_factors[:, i] = np.prod(ratios[:, mask], axis=1)
        
    # Grades
    weighted_grades_u = grades_matrix * all_weights * u_factors
    final_strict = np.sum(weighted_grades_u, axis=1)
    
    weighted_grades_raw = grades_matrix * all_weights
    final_raw = np.sum(weighted_grades_raw, axis=1)
    
    # Strict Fail Rule (< 3.0 -> 0.8 * Raw)
    mask_fail = final_raw < 3.0
    final_grade = np.where(mask_fail, 0.8 * final_raw, final_strict)
    
    if return_details:
        return final_grade, final_raw, u_factors, all_names
    return final_grade, final_raw


In [262]:
# 6. OPTIMIZACIÓN (Objetivo Híbrido: Accuracy + Min FPR)

def objective_function(x_independent, df, config):
    # 1. Reconstruct full weights
    weights_regular = reconstruct_full_weights(x_independent, config)
    
    # 2. Get Ground Truth
    y_true = get_oracle_decisions_general(weights_regular, df, config)
    
    # 3. Calculate Grades
    y_pred, _ = calculate_grade_strict_general(weights_regular, df, config)
    
    # 4. Error Híbrido
    pass_th = 2.95
    # False Negatives (Should Win, Failed)
    err_fn = np.maximum(0, pass_th - y_pred[y_true==1])**2
    
    # False Positives (Should Fail, Won) -> PENALIZED HIGHER
    LAMBDA_FP = 5.0
    err_fp = np.maximum(0, y_pred[y_true==0] - pass_th)**2
    
    return np.sum(err_fn) + (LAMBDA_FP * np.sum(err_fp))

# Setup Variables
reg_items = config_evaluacion['regular_items']
indep_indices = get_independent_indices(config_evaluacion)
indep_items = [reg_items[i] for i in indep_indices]

x0 = np.array([item['peso_sugerido'] for item in indep_items])
bounds = Bounds(
    [item['peso_min'] for item in indep_items],
    [item['peso_max'] for item in indep_items]
)

# Constraints Logic
coeffs = np.zeros(len(indep_items))
name_to_idx = {item['name']: i for i, item in enumerate(indep_items)}

for i, item in enumerate(reg_items):
    if 'correlated_with' in item:
        parent = item['correlated_with']
        factor = item['correlation_factor']
        if parent in name_to_idx:
            idx = name_to_idx[parent]
            coeffs[idx] += factor
    else:
        if item['name'] in name_to_idx:
            idx = name_to_idx[item['name']]
            coeffs[idx] += 1.0

def_item = config_evaluacion['definitory_item']
max_w_final = def_item.get('max_weight', 0.40)
min_w_regular = 1.0 - max_w_final

linear_constraint = LinearConstraint(
    coeffs,
    min_w_regular, 
    0.99
)

print(f"Optimizando (FPR Penalty x5.0)... Restricción Suma Regular: >={min_w_regular:.2f}")

res = minimize(
    objective_function, 
    x0, 
    args=(df_notas, config_evaluacion),
    method='trust-constr',
    bounds=bounds,
    constraints=[linear_constraint],
    options={'verbose': 1}
)

print(f"\nOptimización Exitosa: {res.success}")
opt_indep = res.x
opt_weights_full = reconstruct_full_weights(opt_indep, config_evaluacion)
opt_final_weight = 1.0 - np.sum(opt_weights_full)

print(f"\n{'ITEM':<15} | {'OPTIMIZADO':<10}")
print("-" * 40)
for i, item in enumerate(reg_items):
    print(f"{item['name']:<15} | {opt_weights_full[i]:<10.2%}")
print("-" * 40)
print(f"{def_item['name']:<15} | {opt_final_weight:<10.2%}")


Optimizando (FPR Penalty x5.0)... Restricción Suma Regular: >=0.60
`gtol` termination condition is satisfied.
Number of iterations: 12, function evaluations: 20, CG iterations: 9, optimality: 1.81e-12, constraint violation: 0.00e+00, execution time: 0.075 s.

Optimización Exitosa: True

ITEM            | OPTIMIZADO
----------------------------------------
Quiz_1          | 8.11%     
Tarea_1         | 16.22%    
Quiz_2          | 9.73%     
Tarea_2         | 19.46%    
Quiz_3          | 8.11%     
Tarea_3         | 16.22%    
----------------------------------------
Examen Final    | 22.16%    


In [263]:
# 7. COMPARATIVA FINAL: FPR & OBJECTIVE VALUE

# A. Crear config_evaluacion_optimized (Optimizado)
config_evaluacion_optimized = copy.deepcopy(config_evaluacion)
for i, item in enumerate(config_evaluacion_optimized['regular_items']):
    item['weight'] = float(opt_weights_full[i])
config_evaluacion_optimized['definitory_item']['weight'] = float(opt_final_weight)

# B. Helper para Derivar Pesos
def get_derived_weights(config_in):
    reg_items = config_in['regular_items']
    def_item = config_in['definitory_item']
    weight_map = {}
    
    # Init
    for item in reg_items:
        if 'weight' in item:
            weight_map[item['name']] = item['weight']
        elif 'correlated_with' not in item:
            weight_map[item['name']] = item.get('peso_sugerido', 0.0)
            
    # Propagate
    final_reg_weights = []
    for item in reg_items:
        w = 0.0
        if 'weight' in item:
             w = item['weight']
        elif 'correlated_with' in item:
             parent = item['correlated_with']
             factor = item['correlation_factor']
             if parent in weight_map:
                 w = weight_map[parent] * factor
             else:
                 w = item.get('peso_sugerido', 0.0)
        else:
             w = item.get('peso_sugerido', 0.0)
        weight_map[item['name']] = w
        final_reg_weights.append(w)
    
    final_reg_weights = np.array(final_reg_weights)
    
    # Residual
    if 'weight' in def_item:
        ft_w = def_item['weight']
    else:
        ft_w = 1.0 - np.sum(final_reg_weights)
        if ft_w < 0: ft_w = 0.0
        
    return final_reg_weights, ft_w

# C. Rutina de Reporte (FPR + LOSS)
def calculate_metrics_generic(config_in, df, title="Config"):
    w_reg, w_def = get_derived_weights(config_in)
    
    y_true = get_oracle_decisions_general(w_reg, df, config_in)
    grade_final, _ = calculate_grade_strict_general(w_reg, df, config_in)
    
    pass_th = 2.95
    LAMBDA_FP = 5.0
    
    # 1. FPR Calculation
    fp = ((y_true == 0) & (grade_final >= 2.95)).sum()
    tn = ((y_true == 0) & (grade_final < 2.95)).sum()
    real_negatives = fp + tn
    fpr = fp / (real_negatives + 1e-9)
    
    # 2. Objective Loss Calculation
    err_fn = np.maximum(0, pass_th - grade_final[y_true==1])**2
    err_fp = np.maximum(0, grade_final[y_true==0] - pass_th)**2
    total_loss = np.sum(err_fn) + (LAMBDA_FP * np.sum(err_fp))
    
    print(f"[{title:20}] FPR: {fpr:.2%} (FP={fp}/{real_negatives}) | Loss: {total_loss:8.2f} | Final W: {w_def:.2%}")
    return fpr, total_loss

print("=== COMPARATIVA DE MÉTRICAS ===")
fpr_orig, loss_orig = calculate_metrics_generic(config_evaluacion, df_notas, title="ORIGINAL (Sugerido)")
fpr_opt, loss_opt = calculate_metrics_generic(config_evaluacion_optimized, df_notas, title="OPTIMIZADO")

print(f"\nMejora de FPR: {(fpr_orig - fpr_opt)*100:.2f} puntos porcentuales")
print(f"Reducción de Loss: {loss_orig - loss_opt:.2f}")

def calculate_fpr_from_config(config_opt, df):
    calculate_metrics_generic(config_opt, df, title="Verified Optimized")


=== COMPARATIVA DE MÉTRICAS ===
[ORIGINAL (Sugerido) ] FPR: 0.34% (FP=3/879) | Loss:  1776.21 | Final W: 28.00%
[OPTIMIZADO          ] FPR: 0.57% (FP=5/879) | Loss:  1607.05 | Final W: 22.16%

Mejora de FPR: -0.23 puntos porcentuales
Reducción de Loss: 169.16


In [264]:
# 8. REPORTE VISUAL (FINAL)

# Llamamos a calculate con return_details=True
grade_final, grade_raw, u_factors, item_names = calculate_grade_strict_general(
    opt_weights_full, df_notas, config_evaluacion, return_details=True
)
y_true_final = get_oracle_decisions_general(opt_weights_full, df_notas, config_evaluacion)

df_reporte = df_notas.copy()
df_reporte['Nota_Base'] = grade_raw
df_reporte['Nota_Final'] = grade_final
df_reporte['Oraculo'] = y_true_final

for i, name in enumerate(item_names):
    df_reporte[f'U_{name}'] = u_factors[:, i]

df_reporte['Decision_Nota'] = np.where(df_reporte['Nota_Final'] >= 2.95, 'Ganó', 'Perdió')
df_reporte['Is_False_Positive'] = (df_reporte['Oraculo'] == 0) & (df_reporte['Nota_Final'] >= 2.95)
df_reporte['Is_False_Negative'] = (df_reporte['Oraculo'] == 1) & (df_reporte['Nota_Final'] < 2.95)

ordered_cols = []
for name in item_names:
    ordered_cols.append(name)
    ordered_cols.append(f'U_{name}')
ordered_cols.extend(['Nota_Base', 'Nota_Final', 'Oraculo', 'Decision_Nota', 'Is_False_Positive', 'Is_False_Negative'])

def show_subset(title, dataframe, n=10):
    print(f"\n>>> {title} (Mostrando {len(dataframe)}):")
    if len(dataframe) == 0:
        print("   (No hay casos)")
    else:
        display(dataframe[ordered_cols].head(n))

df_ok = df_reporte[(~df_reporte['Is_False_Positive']) & (~df_reporte['Is_False_Negative'])]
df_ok_gray = df_ok[(df_ok['Nota_Base'] >= 2.5) & (df_ok['Nota_Base'] <= 3.5)]
df_balanced_ok = pd.concat([
    df_ok_gray[df_ok_gray['Decision_Nota'] == 'Ganó'].head(10),
    df_ok_gray[df_ok_gray['Decision_Nota'] == 'Perdió'].head(10)
])

show_subset("Casos Correctos Zona Gris (10 Ganaron / 10 Perdieron)", df_balanced_ok, n=20)
show_subset("Falsos Positivos (Ganó sin merecerlo)", df_reporte[df_reporte['Is_False_Positive']], n=10)
show_subset("Falsos Negativos (Perdió mereciendo ganar)", df_reporte[df_reporte['Is_False_Negative']], n=10)



>>> Casos Correctos Zona Gris (10 Ganaron / 10 Perdieron) (Mostrando 20):


,Quiz_1,U_Quiz_1,Tarea_1,U_Tarea_1,Quiz_2,U_Quiz_2,Tarea_2,U_Tarea_2,Quiz_3,U_Quiz_3,Tarea_3,U_Tarea_3,Examen Final,U_Examen Final,Nota_Base,Nota_Final,Oraculo,Decision_Nota,Is_False_Positive,Is_False_Negative
279,3.66,0.99,3.04,0.99,4.04,0.99,2.96,1.00,4.42,0.99,3.09,0.99,3.06,0.99,3.30,3.26,1,Ganó,False,False
492,3.44,0.90,3.25,0.90,4.31,0.90,3.19,0.90,3.25,0.90,2.71,1.00,3.55,0.90,3.34,3.06,1,Ganó,False,False
697,2.86,0.99,2.97,0.95,3.51,0.94,3.53,0.94,4.30,0.94,3.52,0.94,3.01,0.94,3.33,3.16,1,Ganó,False,False
726,4.35,0.94,3.06,0.94,4.32,0.94,3.41,0.94,3.33,0.94,3.57,0.94,2.83,1.00,3.41,3.25,1,Ganó,False,False
901,3.77,0.91,2.72,1.00,4.25,0.91,3.19,0.91,4.20,0.91,3.59,0.91,3.54,0.91,3.49,3.20,1,Ganó,False,False
908,3.88,1.00,3.39,1.00,4.46,1.00,3.05,1.00,4.48,1.00,3.51,1.00,3.00,1.00,3.49,3.49,1,Ganó,False,False
976,4.27,0.93,2.79,1.00,4.32,0.93,3.24,0.93,3.99,0.93,3.23,0.93,3.61,0.93,3.50,3.28,1,Ganó,False,False
1166,2.97,0.98,2.94,0.99,3.20,0.97,3.52,0.97,4.34,0.97,4.07,0.97,3.29,0.97,3.45,3.36,1,Ganó,False,False
1302,3.78,0.84,3.20,0.84,3.82,0.84,3.86,0.84,4.15,0.84,2.52,1.00,3.41,0.84,3.45,2.96,1,Ganó,False,False
1313,3.73,0.87,3.15,0.87,3.82,0.87,2.62,1.00,4.20,0.87,3.12,0.87,3.47,0.87,3.31,2.96,1,Ganó,False,False



>>> Falsos Positivos (Ganó sin merecerlo) (Mostrando 5):


,Quiz_1,U_Quiz_1,Tarea_1,U_Tarea_1,Quiz_2,U_Quiz_2,Tarea_2,U_Tarea_2,Quiz_3,U_Quiz_3,Tarea_3,U_Tarea_3,Examen Final,U_Examen Final,Nota_Base,Nota_Final,Oraculo,Decision_Nota,Is_False_Positive,Is_False_Negative
16,4.11,0.76,3.65,0.76,3.22,0.76,5.00,0.76,4.18,0.76,4.27,0.76,2.28,1.00,3.75,2.97,0,Ganó,True,False
82,4.50,0.80,3.45,0.80,4.46,0.80,4.85,0.80,4.11,0.80,3.71,0.80,2.41,1.00,3.77,3.13,0,Ganó,True,False
656,3.73,0.80,5.00,0.80,3.57,0.80,3.80,0.80,4.39,0.80,3.53,0.80,2.39,1.00,3.66,3.02,0,Ganó,True,False
1140,4.33,0.76,4.75,0.76,4.01,0.76,4.33,0.76,4.02,0.76,3.76,0.76,2.29,1.00,3.80,3.02,0,Ganó,True,False
1725,3.92,0.75,4.52,0.75,4.21,0.75,4.40,0.75,4.25,0.75,3.91,0.75,2.26,1.00,3.80,2.98,0,Ganó,True,False



>>> Falsos Negativos (Perdió mereciendo ganar) (Mostrando 948):


,Quiz_1,U_Quiz_1,Tarea_1,U_Tarea_1,Quiz_2,U_Quiz_2,Tarea_2,U_Tarea_2,Quiz_3,U_Quiz_3,Tarea_3,U_Tarea_3,Examen Final,U_Examen Final,Nota_Base,Nota_Final,Oraculo,Decision_Nota,Is_False_Positive,Is_False_Negative
0,3.30,0.66,4.99,0.66,4.45,0.66,4.58,0.66,3.59,0.66,1.99,1.00,3.63,0.66,3.82,2.64,1,Perdió,False,True
2,0.01,0.83,2.50,0.00,4.05,0.00,4.29,0.00,3.62,0.00,3.14,0.00,3.84,0.00,3.29,0.01,1,Perdió,False,True
4,4.43,0.06,1.48,0.12,4.31,0.06,0.66,0.27,4.19,0.06,1.65,0.11,3.36,0.06,2.50,2.00,1,Perdió,False,True
5,4.47,0.22,4.70,0.22,3.48,0.22,0.66,1.00,3.83,0.22,4.70,0.22,3.42,0.22,3.42,0.85,1,Perdió,False,True
8,3.87,0.35,2.25,0.46,2.68,0.39,1.55,0.67,3.73,0.35,3.86,0.35,4.95,0.35,3.27,1.28,1,Perdió,False,True
9,4.30,0.16,2.09,0.23,3.45,0.16,4.06,0.16,1.04,0.46,2.33,0.20,2.53,0.19,2.84,2.27,1,Perdió,False,True
10,4.43,0.01,3.32,0.01,3.63,0.01,0.03,0.83,3.92,0.01,2.49,0.01,4.70,0.01,3.02,0.03,1,Perdió,False,True
11,3.83,0.08,3.22,0.08,1.81,0.13,1.04,0.23,1.14,0.21,4.46,0.08,4.67,0.08,3.06,0.29,1,Perdió,False,True
13,3.70,0.18,3.10,0.18,4.33,0.18,2.72,0.20,2.18,0.25,0.84,0.66,3.36,0.18,2.81,2.25,1,Perdió,False,True
15,3.94,0.29,4.41,0.29,3.09,0.29,1.92,0.45,4.13,0.29,1.37,0.63,2.96,0.29,2.92,2.34,1,Perdió,False,True
